In [ ]:
'''
Created By Josh Stokley to create tcl files of RC Walls to be ran through OpenSees
The class provides-
variables:
 self.cyc ........................................ boolean, true if wall will be ran cyclic or not
 self.walldata ................................... large array from Matlab_to_Python that holds variables in reference to wall being built
 self.wallspecimen ............................... unique id of wall
 self.data_file .................................. folder wall will be held in
 self.len_of_element ............................. length of element
 self.doubleEles ................................. boolean, determines if mesh is too coarse
 self.len_of_webBars ............................. determines distance inbetween vertical bars in web
 self.webTruss ................................... location of vertical bars in the web
 self.totLayers .................................. total layers of elements in mesh
 self.blayout .................................... total amount of bars in boundary
 self.wlayout .................................... total amount of bars in web
 self.boundTruss ................................. location of vertical bars in boundary
 self.nodeArray .................................. location of nodes along base
 self.nodesAlongWidth ............................ total amount of nodes along base
 self.totNodes ................................... total nodes in wall
 self.xNodes ..................................... location of nodes in x direction
 self.stc ........................................ shear retention factor
 self.E .......................................... elastic modulus of concrete
 self.gtucc ...................................... fracture energy of concrete in confined region
 self.gtuc ....................................... fracture energy of concrete in unconfined region
 self.gfuc ....................................... crushing energy of concrete in confined region
 self.gfcc ....................................... crushing energy of concrete in unconfined region
 self.epscu_cc ................................... strain at crushing of confined region
 self.epstu_cc ................................... strain at fracture of confined region
 self.epscu_uc ................................... strain at crushing of unconfined region
 self.epstu_uc ................................... strain at fracture of confined region
 self.matTag ..................................... updated material tag for materials
 self.bevertTag .................................. vertical bar steel material tag in boundary
 self.vertTag .................................... vertical bar steel material tag in web
 self.lzTag ...................................... concrete elastic material tag
 self.horTag ..................................... horizontal steel material tag
 self.hoop ....................................... boolean if wall has hoop steel
 self.hoopTag .................................... hoop steel material tag
 self.confsecTag ................................. boundary shell element tag
 self.unconfsecTag ............................... web shell element tag
 self.confsecTagElastic .......................... elastic boundary shell element tag
 self.unConfsecTagElastic ........................ elastic web shell element tag
 self.maxEle ..................................... total amount of shell elements
 self.trussele ................................... total elements (shell + truss elements)
 self.lastRow .................................... top row of nodes on wall
 
 
methods:
 def __init__(self,authorIndex,cyc) .................. constructor: imports wall index and if the wall will be ran cyclic or not. creates folder in reference to current date
 def section1(self) .................................. Initialization of the model. The degrees of freedom and the variables that carry uncertainty are defined
 def section2(self) .................................. Defines nodal locations and elements. Nodes are placed at the locations of the vertical bars along the length of the wall. If the ratio of the length of the wall to the number of elements is too coarse of a mesh, 
                                                       additional nodes are placed inbetween the bars. The height of each element is equal to the length of the nodes in the boundary to create square elements up the wall.
 def section3(self) .................................. Defines material models and their variables. The crushing energy and fracture energy are calculated and wrote to the .tcl file. The material models are defined.
                                                       minMax wrappers are applied to the steel so that if the steel strain compresses more than the crushing strain of the concrete or exceeds the ultimate strain of the steel multiplied by the steel rupture ratio, 
                                                       the stress will go to 0.
 def section4(self) .................................. Defines the continuum shell model. The shell element is split up into multiple layers of the cover concrete, transverse steel, and core concrete. The cover concrete thickness is defined in the database, 
                                                       the transverse steel thickness is calculcated as: total layers of transverse steel multiplied by the area of the steel divided by the height of the wall. The total thickness of the wall is defined in the database so after the cover concrete and steel thicknesses are subtracted, 
                                                       the core concrete takes up the rest.
 def section5(self) .................................. Elements are defined using ShellMITC4 and truss elements are defined after.
 def section6(self) .................................. The bottom row of nodes are fixed in all degrees of freedom.
 def section7(self) .................................. Defines recorders. The first two recorders capture the force reactions in the x-direction of the bottom row of nodes and the displacements in the x-direction of the top row of nodes. These recorders will be used to develop load-displacement graphs. 
                                                       The next eight recorders capture stress and strain of the four gauss points in the middle concrete fiber of all the elements and store them in an xml file. These recorders will be used to develop stress and strain profile movies, give insight to how the wall is failing, 
                                                       and how the cross section is reacting. The last two recorders capture the stress and strain of all the truss elements. These will be used to determine when the steel fails and when the yield strength is reached.
 def section8(self) .................................. Defines and applies the gravity load of the wall. The axial load of the wall is defined in the database and distributed equally amongst the top nodes and a static analysis is conducted to apply a gravity load to the wall.
 def section9(self) .................................. Defines the analysis of the wall The experimental displacement recording of the wall is defined in the database and the peak displacement of each cycle is extracted. If the effective height of the wall is larger than the measured height, 
                                                       a moment is calculated from that difference and uniformly applied in the direction of the analysis to each of the top nodes.
 def creatScript(self) ............................... Runs all sections to create file and creates a reference file with variables to be used in post processing: nodes along width, total truss nodes, total nodes, total elements, displacement peaks, fracture strength of concrete, 
                                                       total layers, wall unique id, filepath, tclfile path
'''







%run Matlab_to_Python.ipynb
import math
class createTclFile():
    def __init__(self,authorIndex,cyc):
        self.cyc = cyc
     
        self.authorIndex = authorIndex
        data = grabValues(authorIndex)
        self.Walldata = data.walldata
        self.today = datetime.now()
        if not os.path.exists('tcl_files'):
            os.makedirs('tcl_files') 

       # self.df = pd.read_excel("WallDatabase.xlsx",sheet_name='US Units')
        self.wallspecimen = self.Walldata[1]
        print(self.wallspecimen)

        self.date = self.today.strftime("%y%m%d%H%M%S")
        self.data_file = self.today.strftime("%y%m%d%H")
        if not os.path.exists('tcl_files/tcl_files_'+self.data_file + '/' + self.wallspecimen + '_'+ self.date):
            os.makedirs('tcl_files/tcl_files_'+self.data_file + '/' + self.wallspecimen + '_'+ self.date) 

        if not os.path.exists('tcl_files/tcl_files_'+self.data_file + '/' + self.wallspecimen + '_'+ self.date + '/'+ self.date + '.tcl'):
            self.f = open("tcl_files/tcl_files_"+self.data_file + '/'+self.wallspecimen + '_'+ self.date + '/'+ self.date+ '.tcl',"w+")
            self.r = open("tcl_files/tcl_files_"+self.data_file + '/'+self.wallspecimen + '_'+ self.date + '/'+ self.date+ 'referenceFile.txt',"w+")
        
        self.filepathtcl = "tcl_files/tcl_files_"+self.data_file + '/'+self.wallspecimen + '_'+ self.date + '/'+ self.date+ '.tcl'
        self.filepath = "tcl_files/tcl_files_"+self.data_file + '/'+self.wallspecimen + '_'+ self.date + '/'
        self.filename = self.wallspecimen + '_'+ self.date
        self.createScript()
        
    def section1(self): #Section 1: Initialize
        
        self.f.write("""
wipe

#####################
puts "Initialization"
#####################

model BasicBuilder -ndm 3 -ndf 6;
""")
        self.f.write("#Wall Specimen: " + str(self.wallspecimen) + '\n')
        self.f.write("set stc 0.056\n")
        self.f.write("set shr 1\n")
        
        self.f.write('#if Failure mode is CB:\n')
        self.f.write("pset rev 2.56\n")
        #self.f.write("set mult 1\n")
        self.f.write('#if Failure mode is BR:\n')
        self.f.write("pset srs 0.3\n")
        

    def section2(self):  #Section 2: Nodes
        
        self.f.write('''


#####################
puts "Define Nodes"
#####################


''')

        if (self.Walldata[6]<self.Walldata[5]):
            height = self.Walldata[6]
            print(2)
       
        else:
            height = self.Walldata[5]
            print(1)
        

        
        loaded = io.loadmat('WallData.mat')
        data = loaded['WallData']
        numOfBars = len(data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['Layers'][0,0]['barType'])
        barType = []
        trussbarType = []
        for ii in range(numOfBars):
            loc = str(data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['Layers'][0,0]['barType'][ii,0]) #grabbing bar type
            loc = loc.replace("[", "") #removing brackets
            loc = loc.replace("]", "")
            loc = loc.replace("'", "") #removing quotes
            barType.append(loc) 
            trussbarType.append(loc) 

        zLoc = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['Layers'][0,0]['zLoc']

        locations = []
        lengthOfcs = float(zLoc[-1][0])*2
        for ii in range(len(zLoc)):
            if ii == 0:
                locations.append(0)
            else:
                locations.append(round(abs(float(zLoc[0][0])) + float(zLoc[ii][0]),2))
        minD = 100
        for ii in range(1,len(locations)):
            if (locations[ii]-locations[ii-1])<minD:
                minD = (locations[ii]-locations[ii-1])
        maxD = 0
        for ii in range(1,len(locations)):
            if (locations[ii]-locations[ii-1])>maxD:
                maxD = (locations[ii]-locations[ii-1])
                maxD = round(maxD,2)

        firstMax = maxD

        self.len_of_element = round(firstMax,2)
        findLength = False
        self.nodeArray = []
        for ii in locations:
            self.nodeArray.append(ii)
        maxD1 = 0
       
        bound = 0
        for ii in range(len(barType)):
            if barType[ii] == 'b' or barType[ii] == 'ba':
                bound +=1
         
        bound = bound/2
        
        if bound <3:
            loc = []
            ind = []
            for ii in range(1,len(barType)):
                if (barType[ii] =='b' and barType[ii-1] =='b') or (barType[ii] =='ba' and barType[ii-1] =='b') or (barType[ii] =='b' and barType[ii-1] =='ba'):
                    loc.append(self.nodeArray[ii-1]+(self.nodeArray[ii]-self.nodeArray[ii-1])/2)
                    ind.append(ii)
            for ii in range(len(ind)):
                barType.insert(ii+ind[ii],'b')
                    
            for ii in range(len(loc)):
                self.nodeArray.append(round(loc[ii],2))
        self.nodeArray.sort()
     
        findArray = False
        while (height/self.nodeArray[1])<40:
            loc = []
            ind = []
            for ii in range(1,len(barType)):
                if barType[ii] =='b' and barType[ii-1] =='b':
                    loc.append(self.nodeArray[ii-1]+(self.nodeArray[ii]-self.nodeArray[ii-1])/2)
                    ind.append(ii)
            for ii in range(len(ind)):
                barType.insert(ii+ind[ii],'b')
                    
            for ii in range(len(loc)):
                self.nodeArray.append(round(loc[ii],2))
            self.nodeArray.sort()  
        
            locMax = []
            for ii in range(len(barType)):
                if (barType[ii] == 'w' and barType[ii-1] == 'w') or (barType[ii] == 'b' and barType[ii-1] == 'w') or (barType[ii-1] == 'b' and barType[ii] == 'w'):
                    locMax.append(self.nodeArray[ii]-self.nodeArray[ii-1])
                    
                
        sqrEle = False
        while sqrEle == False:
            loc = []
            ind = []
            for ii in range(len(barType)):
                if (barType[ii] == 'w' and barType[ii-1] == 'w') or (barType[ii] == 'b' and barType[ii-1] == 'w') or (barType[ii-1] == 'b' and barType[ii] == 'w') or (barType[ii] == 'ba' and barType[ii-1] == 'w') or (barType[ii-1] == 'ba' and barType[ii] == 'w'):
                    loc.append(self.nodeArray[ii-1]+(self.nodeArray[ii]-self.nodeArray[ii-1])/2)
                    ind.append(ii)
#
            for ii in range(len(ind)):
                barType.insert(ii+ind[ii],'w')
                    
            for ii in range(len(loc)):
                self.nodeArray.append(round(loc[ii],2))
            self.nodeArray.sort()  
            
            locMax = []
            for ii in range(len(barType)):
                if (barType[ii] == 'w' and barType[ii-1] == 'w') or (barType[ii] == 'b' and barType[ii-1] == 'w') or (barType[ii-1] == 'b' and barType[ii] == 'w'):
                    locMax.append(self.nodeArray[ii]-self.nodeArray[ii-1])
             
            locMax = max(locMax)
            
            nodeMax = 1
            for ii in range(1,len(self.nodeArray)):
                if (self.nodeArray[ii]-self.nodeArray[ii-1]) > nodeMax:
                    nodeMax = self.nodeArray[ii]-self.nodeArray[ii-1]

            if self.nodeArray[1]/nodeMax > 0.75:
                sqrEle = True

        self.barType = barType    
        print(self.nodeArray)
        self.len_of_element = self.nodeArray[1]
        self.totLayers = int(height/self.len_of_element)+1 
        loaded = io.loadmat('WallData.mat')
        data = loaded['WallData']
        check = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen].dtype.fields
        self.oppo = False
        if 'modSR_e' in check:
            self.oppo = True
        self.blayout = []
        self.wlayout =[]
        self.balayout = []
        print(zLoc)
        print(trussbarType)
        for ii in range(len(trussbarType)):
            if trussbarType[ii] == 'w':
                self.wlayout.append(zLoc[ii][1])
            elif trussbarType[ii]== 'b':
                self.blayout.append(zLoc[ii][1])
            else:
                self.balayout.append(zLoc[ii][1])
        if self.oppo == True:
            self.totBound = self.balayout+self.blayout
        else:
            self.totBound = self.blayout
        self.boundTruss = []
        self.boundaTruss = []
        self.webTruss = []

        k = 0
        for ii in range(len(self.nodeArray)):
            if self.nodeArray[ii] == locations[k]:
                if trussbarType[k] == 'b':
                    self.boundTruss.append(ii+1)
                    k+=1
                elif trussbarType[k] == 'w':
                    self.webTruss.append(ii+1)
                    k+=1
                elif trussbarType[k] == 'ba':
                    self.boundaTruss.append(ii+1)
                    k+=1

        

        self.nodesAlongWidth = len(self.nodeArray)
        node = 1         
        self.xN = []
        self.yN = []
        y = 0            
        self.totLayers = 0
        up = self.len_of_element

        nxt = False
        layers = False

        trans = height/2
        for ii in range(len((self.nodeArray))):
            (self.nodeArray[ii]) = round((self.nodeArray[ii]),2)
            y = 0
        while layers == False:

            locY = height - up
            locY1 = height - 2*up
            if math.isclose(y,locY,abs_tol = 2):#y == y-up:
                y = round(height,2)

            elif math.isclose(y,locY1,abs_tol = 2):
                y = y-up
                y = round(y+(height-y)/2,2)
            elif y > height-trans and nxt == False:

                up = 1.25*up
                nxt = True

            for ii in range(0,len(self.nodeArray)):
                self.f.write('node ' + str(node) + ' ' + str(self.nodeArray[ii]) + ' ' + str(y) + ' 0\n')
                self.xN.append(self.nodeArray[ii])
                self.yN.append(y)
                node +=1    
            y += up
            self.totLayers +=1
            y = round(y,2)
            if y > height:
                layers = True
            self.f.write('\n')
        self.totNodes = node-1
        
        self.nodes_along_width = len(self.nodeArray)
        self.xNodes = []
        for ii in range(1,self.nodes_along_width+1):
            self.xNodes.append(ii)

        #Node 1 will be at boundary Element length - cover
        #Rest of nodes will be decided by where the vertical bars are
        #Take length bewteen boundary nodes, that is the height of integration points
        #Put each row of nodes in an array
        
    def section3(self): #Section 3: Materials
    
        #Section 3: Materials
        self.f.write("""


#####################
puts "Writing Materials"
#####################


""")
        E = (57000*(self.Walldata[40]*1000)**(1/2))/1000
        if self.Walldata[34] == 0:
            self.hoop = False
        else:
            self.hoop = True
            
        loaded = io.loadmat('WallData.mat')
        data = loaded['WallData']
        kcBound = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['modSR'][0,0]['kc'][0,0]
        
        if kcBound == 1:
            self.confind = False
            eps1 = 0.002
        else:
            eps1 = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['modSR'][0,0]['eps1'][0,0]
            self.confind = True
        check = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen].dtype.fields
        print(check)
        if 'webmodSR' in check:
            kcWeb = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['webmodSR'][0,0]['kc'][0,0]
            epsweb = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['webmodSR'][0,0]['eps1'][0,0]



        else:
            epsweb = 2*self.Walldata[40]/E
            kcWeb = 1
        kcbA = 0   
        if 'modSR_e' in check:
            eps3 = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['modSR_e'][0,0]['eps1'][0,0]
            kcbA = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['modSR_e'][0,0]['kc'][0,0]
            self.oppo = True
        if str(kcBound) == 'nan':
            epsweb = 2*self.Walldata[40]/E
            kcBound = 1
            #print(kcBound)
        if str(kcWeb) == 'nan':
            epsweb = 2*self.Walldata[40]/E
            kcWeb = 1
        if self.confind == False:
            epsweb = eps1
        self.kcBound = kcBound    
        if kcBound == 1 and kcWeb == 1:
            fcB = self.Walldata[40]
            fcW = self.Walldata[40]
            epc0B,epc0W = -eps1,-epsweb
        else:
            fcB = self.Walldata[40]*kcBound
            fcW = self.Walldata[40]*kcWeb
            epc0B,epc0W = -eps1,-epsweb
        
        if self.oppo == True:
            if kcbA == 0:
                fcBa = self.Walldata[40]
            else:
                fcBa = self.Walldata[40]*kcbA
            epc0Ba = -eps3
            
        #print(fcW)
        print(epsweb)    
        self.stc = '$stc' #shear retention factor
        self.E = (57000*(fcW*1000)**(1/2)) #Elastic modulus of concrete
        self.gtcc = abs((0.174*(.5)**2-0.0727*.5+0.149)*((fcB*1000)/1450)**0.7) #tensile energy of confined
        self.gtuc = abs((0.174*(.5)**2-0.0727*.5+0.149)*((fcW*1000)/1450)**0.7) # tensile energy of unconfined
        if self.oppo == True:
            self.gtcca = abs((0.174*(.5)**2-0.0727*.5+0.149)*((fcBa*1000)/1450)**0.7)
        #epsc0 is grabbed from .mat file
        #epscu = 2*Gf/[(beta + 1)*fc*Le] + epsc0*(beta+1)/2
        #epstu = ft/Ec + 2*Gt/(Le*ft)
        
        #gtuc = (0.174*(0.5)**2-0.0727*0.5+0.149)*((fc*1000)/1450)**0.7
        #E = 57000*sqrt(fc)
        #6.89476 converts fcW [ksi] to fcW [N/mm] and 5.71015 converts [N/mm] to [pound/in]
        #self.gfuc = 8.8*(fcW*6.89476)**(1/2)*5.710147 #crushing energy of unconfined
        #self.gfcc = 2.2*self.gfuc #crushing energy of confined

        self.gfuc = 8.8*(fcW*6.89476)**(1/2)*5.71015
        self.f.write('set gfuc '+str(self.gfuc)+'\n')
        if self.confind == False:
            self.gfcc = self.gfuc
            self.f.write('set gfcc '+str(self.gfuc)+'\n')
        
        else:
            self.gfcc = 2.2*self.gfuc
            self.f.write('set gfcc [expr $rev*$gfuc]\n')

                
        self.f.write('set epscu_uc [expr -1*(2*$gfuc/((0.2+1)*' +str(fcW*self.len_of_element*1000) + ')+'+str(epsweb)+'*0.6)]\n')
        self.f.write('set epscu_cc [expr -1*(2*$gfcc/((0.2+1)*' +str(fcB*self.len_of_element*1000) + ')+'+str(eps1)+'*0.6)]\n')        
        if self.oppo == True:
            self.gfcca = 2.2*self.gfuc
            self.f.write('set gfuca '+ str(self.gfuc)+'\n')
            self.f.write('set gfcca [expr $rev*$gfuca]\n')       
            self.f.write('set epscu_cca [expr -1*(2*$gfcca/((0.2+1)*' +str(fcBa*self.len_of_element*1000) + ')+'+str(eps3)+'*0.6)]\n')
            self.epscu_cca = -1*(2*self.gfcca/((0.2 + 1)*fcBa*1000*self.len_of_element) + 0.004*0.6)
            self.epstu_cca = (self.Walldata[42]*1000)/self.E+2*self.gtcca/(self.len_of_element*self.Walldata[42]*1000)   
            ccaProps = [fcBa,self.Walldata[42],-fcBa*0.2,epc0Ba,'$epscu_cc',self.epstu_cca,'$stc',self.E/2000]
            ccaProps = (list(map(str, ccaProps)))       
            bayield = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['beSteel_e'][0,0]['fy'][0,0]
            befu = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['beSteel_e'][0,0]['fu'][0,0]
            beUlt = data[0,0]['SectionAnalysis'][0,0][self.wallspecimen][0,0]['beSteel_e'][0,0]['eps_ult'][0,0]
            beasteelProps = [bayield,29000,(befu-bayield)/(beUlt-self.Walldata[17])/29000]
            beasteelProps = (list(map(str, beasteelProps)))
            
        self.epscu_cc = -1*(2*self.gfcc/((0.2 + 1)*fcB*1000*self.len_of_element) + 0.004*0.6)

        self.epstu_cc = (self.Walldata[42]*1000)/self.E+2*self.gtcc/(self.len_of_element*self.Walldata[42]*1000)
        
        self.epscu_uc = -1*(2*self.gfuc/((0.2 + 1)*fcW*1000*self.len_of_element) + 0.002*0.6)
        
        self.epstu_uc = (self.Walldata[42]*1000)/self.E+2*self.gtuc/(self.len_of_element*self.Walldata[42]*1000)   
        
        #print(self.epscu_uc)
        #print('')
        #props  =  fc, ft,               fcu,     epsc0,  epscu,    epstu,          stc,  shearmod
        ccProps = [fcB,self.Walldata[42],-fcB*0.2,epc0B,'$epscu_cc',self.epstu_cc,'$stc',self.E/2000]
        ccProps = (list(map(str, ccProps)))

        ucProps = [fcW,self.Walldata[42],-(fcW*0.2),epc0W,'$epscu_uc',self.epstu_uc,'$stc',self.E/2000]
        ucProps = (list(map(str, ucProps)))

        #props =      fy,                 E,                        b
        hsteelProps =[self.Walldata[30],29000,(self.Walldata[31]-self.Walldata[30])/(self.Walldata[33]-self.Walldata[32])/29000]
        hsteelProps = (list(map(str, hsteelProps)))
        

        if self.hoop == False:
            var = 1
        else:
            hoopsteelProps = [self.Walldata[36],29000,(self.Walldata[37]-self.Walldata[36])/(self.Walldata[33]-self.Walldata[38])/29000]
            hoopsteelProps = (list(map(str, hoopsteelProps)))
    
        websteelProps = [self.Walldata[23],29000,(self.Walldata[24]-self.Walldata[23])/(self.Walldata[26]-self.Walldata[25])/29000]
        websteelProps = (list(map(str, websteelProps)))
        
        besteelProps = [self.Walldata[15],29000,(self.Walldata[16]-self.Walldata[15])/(self.Walldata[18]-self.Walldata[17])/29000]
        besteelProps = (list(map(str, besteelProps)))

        self.matTag = 1

        self.f.write('#Confined Concrete\n') 
        self.f.write('nDMaterial PlaneStressUserMaterial  ' + str(self.matTag) + '  40  7  ' + ccProps[0] + '  '+ ccProps[1] + '  '+ ccProps[2] + '  '+ ccProps[3] + '  '+ ccProps[4] + '  '+ ccProps[5] + '  '+ ccProps[6] + '\n' )
        self.matTag += 1
        self.f.write('\n') 
        self.f.write('nDMaterial PlateFromPlaneStress   ' + str(self.matTag) + '   ' +  str(self.matTag - 1) + '  ' + ccProps[7] + '\n')
        self.conTag = self.matTag
        self.matTag += 1

        if self.oppo == True:
            self.f.write('\n')
            self.f.write('#Confined Concrete opposite side\n') 
            self.f.write('nDMaterial PlaneStressUserMaterial  ' + str(self.matTag) + '  40  7  ' + ccaProps[0] + '  '+ ccaProps[1] + '  '+ ccaProps[2] + '  '+ ccaProps[3] + '  '+ ccaProps[4] + '  '+ ccaProps[5] + '  '+ ccaProps[6] + '\n' )
            self.matTag += 1
            self.f.write('\n') 
            self.f.write('nDMaterial PlateFromPlaneStress   ' + str(self.matTag) + '   ' +  str(self.matTag - 1) + '  ' + ccaProps[7] + '\n')
            self.conaTag = self.matTag
            self.matTag += 1
    
        self.f.write('\n') 
        self.f.write('#Unconfined Concrete\n') 
        self.f.write('nDMaterial PlaneStressUserMaterial  ' + str(self.matTag) + '  40  7  ' + ucProps[0] + '  '+ ucProps[1] + '  '+ ucProps[2] + '  '+ ucProps[3] + '  '+ ucProps[4] + '  '+ ucProps[5] + '  '+ ucProps[6] + '\n' )
        self.matTag += 1
        self.f.write('\n') 
        self.f.write('nDMaterial PlateFromPlaneStress   ' + str(self.matTag) + '   ' +  str(self.matTag - 1) + '  ' + ucProps[7] + '\n')
        self.unconTag = self.matTag
        self.matTag += 1

        self.f.write('\n') 
        self.f.write('#Horizontal Steel\n') 
        self.f.write('uniaxialMaterial Steel02  ' + str(self.matTag) + '  ' + hsteelProps[0] + '  ' + hsteelProps[1] + '  ' + '[expr $shr*'+str(hsteelProps[2]) + ']' +  '  20  0.925  0.15  ' + '\n')
        self.horTag =self.matTag
        self.matTag +=1

        if self.hoop == False:
            localvar = True
        else:
            self.f.write('\n') 
            self.f.write('#Hoop Steel\n') 
            self.f.write('uniaxialMaterial Steel02  ' + str(self.matTag) + '  ' + hoopsteelProps[0] + '  ' + hoopsteelProps[1] + '  ' + '[expr $shr*'+str(hoopsteelProps[2]) + ']' +  '  20  0.925  0.15  ' + '\n')
            self.hoopTag = self.matTag
            self.matTag +=1


        self.f.write('\n') 
        self.f.write('#Vertical Boundary Steel\n') 
        self.f.write('uniaxialMaterial Steel02  ' + str(self.matTag) + '  ' + besteelProps[0] + '  ' + besteelProps[1] + '  ' + '[expr $shr*'+str(besteelProps[2]) + ']' +  '  20  0.925  0.15  ' + '\n')
        self.bevertTag = self.matTag
        self.matTag +=1
        
        if self.oppo == True:
            self.f.write('\n') 
            self.f.write('#Vertical Boundary Steel Opposite Side\n') 
            self.f.write('uniaxialMaterial Steel02  ' + str(self.matTag) + '  ' + beasteelProps[0] + '  ' + beasteelProps[1] + '  ' + '[expr $shr*'+str(beasteelProps[2]) + ']' +  '  20  0.925  0.15  ' + '\n')
            self.beavertTag = self.matTag
            self.matTag +=1       
            
        self.f.write('\n') 
        self.f.write('#Vertical web Steel\n') 
        self.f.write('uniaxialMaterial Steel02  ' + str(self.matTag) + '  ' + websteelProps[0] + '  ' + websteelProps[1] + '  ' + '[expr $shr*'+str(websteelProps[2]) + ']' +  '  20  0.925  0.15  ' + '\n')
        self.vertTag = self.matTag
        self.matTag +=1


        self.f.write('\n') 
        self.f.write('nDMaterial ElasticIsotropic  ' + str(self.matTag) + '  ' + str(self.E/100) + ' 0.2   \n')
        self.lzTag = self.matTag
        self.matTag +=1


        self.f.write('\n')

        
        if self.hoop == False:
            localvar = True
        else:
            self.f.write('\n')



        self.f.write('\n')
        self.f.write('uniaxialMaterial MinMax ' + str(self.matTag) + ' ' + str(self.vertTag)+ ' -min ' + str(ucProps[4]) + ' -max [expr $srs*' + str(self.Walldata[26]*2)+ ']\n')
        self.vertTag = self.matTag
        self.matTag +=1
        
        self.f.write('\n')
        self.f.write('uniaxialMaterial MinMax ' + str(self.matTag) + ' ' + str(self.bevertTag)+ ' -min ' + str(ccProps[4]) + ' -max [expr $srs*' + str(self.Walldata[18]*2)+ ']\n')
        self.bevertTag = self.matTag
        self.matTag +=1

        if self.oppo == True:
            self.f.write('\n')
            self.f.write('uniaxialMaterial MinMax ' + str(self.matTag) + ' ' + str(self.beavertTag)+ ' -min ' + str(ccaProps[4]) + ' -max [expr $srs*' + str(beUlt*2)+ ']\n')
            self.beavertTag = self.matTag  
            self.matTag +=1

        self.f.write('\n') 
        self.f.write('nDMaterial PlateRebar  ' + str(self.matTag) + '  '+ str(self.horTag) + '  ' +  ' 0   \n')
        self.hzrebar = self.matTag
        self.matTag +=1

        if self.hoop == False:
            localvar = True
        else:
            self.f.write('\n') 
            self.f.write('nDMaterial PlateRebar  ' + str(self.matTag) + '  '+ str(self.hoopTag) + '  ' +  ' 0   \n')
            self.hzrebar2 = self.matTag
            self.matTag +=1

            
    def section4(self): #Section 4: Shell Elements
        
        self.f.write('''


#####################
puts "Define Shells"
#####################


''')

        self.secTag = 1
        self.f.write('#Boundary region \n')

        if self.hoop == False:
            nlayer = True

            #Boundary
            coverthic = self.Walldata[8]
            horthicCon = (self.Walldata[5]/self.Walldata[29]+1)*((self.Walldata[28]/2)**2*m.pi)/self.Walldata[5]
            conthic = (self.Walldata[3]-2*coverthic-2*horthicCon)

            #Web
            unconthic = conthic
        else:
            nlayer = False

            #Boundary
            coverthic = self.Walldata[8]
            horthicCon = (self.Walldata[5]/self.Walldata[29]+1)*((self.Walldata[28]/2)**2*m.pi)/self.Walldata[5]
            horthicCon2 = (self.Walldata[5]/self.Walldata[35]+1)*((self.Walldata[34]/2)**2*m.pi)/self.Walldata[5]
            conthic = (self.Walldata[3]-2*coverthic-2*horthicCon-2*horthicCon2)

            #Web
            unconthic = (self.Walldata[3]-2*coverthic-2*horthicCon)


        if nlayer == True:
            self.f.write('section LayeredShell  ' + str(self.secTag) + '  ' +str(5)+ ' ' +str(self.unconTag) + '  ' + str(coverthic) + '  '+ str(self.hzrebar) + '  '
                    + str(horthicCon) + '  '+ str(self.conTag) + '  '+ str(conthic) + '  '+ str(self.hzrebar) + '  '
                    + str(horthicCon) + '  '+ str(self.unconTag) + '  ' + str(coverthic) + '  ' +  '\n')
        else:
            self.f.write('section LayeredShell  ' + str(self.secTag) + '  ' +str(5)+ '  '+ str(self.unconTag) + '  ' + str(coverthic) + 
                    '  '+ str(self.hzrebar2) + '  ' +  '  '+ str(horthicCon2)+ '  '+ str(self.conTag) + '  '+ str(conthic) + '  '+str(self.hzrebar2) + '  '+ str(horthicCon2)+ '  ' 
                      + str(self.unconTag) + '  ' + str(coverthic) + '  ' +  '\n')
        self.confsecTag = self.secTag
        self.secTag +=1
    
    
        if self.oppo == True:
            self.f.write('#Boundary region Opposite \n')
            self.f.write('section LayeredShell  ' + str(self.secTag) + '  ' +str(5)+ ' ' +str(self.unconTag) + '  ' + str(coverthic) + '  '+ str(self.hzrebar) + '  '
                    + str(horthicCon) + '  '+ str(self.conaTag) + '  '+ str(conthic) + '  '+ str(self.hzrebar) + '  '
                    + str(horthicCon) + '  '+ str(self.unconTag) + '  ' + str(coverthic) + '  ' +  '\n')
            self.confasecTag = self.secTag
            self.secTag +=1
        
        self.f.write('#Web region \n')
        self.f.write('section LayeredShell  ' + str(self.secTag) + '  ' +str(5)+' ' + str(self.unconTag) + '  ' + str(coverthic) + '  '+ str(self.hzrebar) + '  '
                + str(horthicCon) + '  '+ str(self.unconTag) + '  '+ str(unconthic) + '  '+ str(self.hzrebar) + '  '
                + str(horthicCon) + '  '+ str(self.unconTag) + '  ' + str(coverthic) + '  ' +  '\n')
        self.unConfsecTag = self.secTag
        self.secTag +=1
        
        self.f.write('#Boundary region Elastic\n')
        if nlayer == True:
            self.f.write('section LayeredShell  ' + str(self.secTag) + '  ' +str(5)+ ' ' +str(self.lzTag) + '  ' + str(coverthic) + '  '+ str(self.hzrebar) + '  '
                    + str(horthicCon) + '  '+ str(self.lzTag) + '  '+ str(conthic) +'  '+ str(self.hzrebar) + '  '
                    + str(horthicCon) + '  '+ str(self.lzTag) + '  ' + str(coverthic) + '  ' +  '\n')
        else:
            self.f.write('section LayeredShell  ' + str(self.secTag) + '  ' +str(5)+ '  '+ str(self.lzTag) + '  ' + str(coverthic) + 
                    '  '+ str(self.hzrebar2) + '  ' +  '  '+ str(horthicCon2)+ '  '+ str(self.lzTag) + '  '+ str(conthic) +  '  '+str(self.hzrebar2) + '  '+ str(horthicCon2)+ '  ' 
                      + str(self.lzTag) + '  ' + str(coverthic) + '  ' +  '\n')
        self.confsecTagElastic = self.secTag
        self.secTag +=1
    

        self.f.write('#Web region Elastic \n')
        self.f.write('section LayeredShell  ' + str(self.secTag) + '  ' +str(5)+' ' + str(self.lzTag) + '  ' + str(coverthic) + '  '+ str(self.hzrebar) + '  '
                + str(horthicCon) + '  '+ str(self.lzTag) + '  '+ str(unconthic) +  '  '+ str(self.hzrebar) + '  '
                + str(horthicCon) + '  '+ str(self.lzTag) + '  ' + str(coverthic) + '  ' +  '\n')
        self.unConfsecTagElastic = self.secTag
    
        
    def section5(self): #Section 5: Elements
 
        self.f.write('''


#####################
puts "Define Elements"
#####################


''')
       
        node = 1 
        ele = 1
        self.maxEle = (self.nodes_along_width-1)*(self.totLayers-1)
        topEles = self.maxEle - (self.nodes_along_width-1)*2
        jj = 0
        print(self.boundTruss)
        print(self.webTruss)
        print(self.boundaTruss)

        elasticZone = int(0.8*self.totLayers)
        if elasticZone < 2:
            elasticZone = 2
        for ii in range(self.totLayers-1):
            if ii<elasticZone:
                for kk in range(1,self.nodes_along_width):
                    if (self.barType[kk] == 'b' and self.barType[kk-1] == 'b') or (self.barType[kk] == 'ba' and self.barType[kk-1] == 'b') or (self.barType[kk] == 'b' and self.barType[kk-1] == 'ba'):
                        num = self.confsecTag
                    elif (self.barType[kk] == 'w' and self.barType[kk-1] == 'w') or (self.barType[kk] == 'w' and self.barType[kk-1] == 'b') or (self.barType[kk] == 'b' and self.barType[kk-1] == 'w') or (self.barType[kk] == 'ba' and self.barType[kk-1] == 'w') or (self.barType[kk] == 'w' and self.barType[kk-1] == 'ba'):
                        num = self.unConfsecTag                
                
            
            
            
                    self.f.write('element ShellDKGQ ' + str(ele) + ' ' + str(node) + ' ' + str(node+1)+ ' ' + str(node+self.nodes_along_width+1)+ ' ' + str(node+self.nodes_along_width) + ' ' + str(num) + '\n')
                    node+=1
                    ele +=1
                    jj+=1
                    if jj == (self.nodes_along_width-1):
                        node+=1
                        jj = 0
                        self.f.write('\n') 
            else:         
                for kk in range(1,self.nodes_along_width):
                    if (self.barType[kk] == 'b' and self.barType[kk-1] == 'b') or (self.barType[kk] == 'ba' and self.barType[kk-1] == 'b') or (self.barType[kk] == 'b' and self.barType[kk-1] == 'ba'):
                        num = self.confsecTagElastic
                    elif (self.barType[kk] == 'w' and self.barType[kk-1] == 'w') or (self.barType[kk] == 'w' and self.barType[kk-1] == 'b') or (self.barType[kk] == 'b' and self.barType[kk-1] == 'w') or (self.barType[kk] == 'ba' and self.barType[kk-1] == 'w') or (self.barType[kk] == 'w' and self.barType[kk-1] == 'ba'):
                        num = self.unConfsecTagElastic
                
                    
                    self.f.write('element ShellDKGQ ' + str(ele) + ' ' + str(node) + ' ' + str(node+1)+ ' ' + str(node+self.nodes_along_width+1)+ ' ' + str(node+self.nodes_along_width) + ' ' + str(num) + '\n')
                    node+=1
                    ele +=1
                    jj+=1
                    if jj == (self.nodes_along_width-1):
                        node+=1
                        jj = 0
                        self.f.write('\n') 

       
        
        node = 1
        element = ele
        self.totEle = ele-1
        for jj in range(1,self.nodes_along_width+1):
            
            if jj in self.boundTruss:
                node = jj
                area = (((self.Walldata[13])/2)**2*m.pi*int(self.blayout[0]))
                steel_mat = self.bevertTag
    
            elif jj in self.webTruss:
                node =jj
                area = (((self.Walldata[21])/2)**2*m.pi*int(self.wlayout[0]))
                steel_mat = self.vertTag
                
            elif len(self.boundaTruss) > 0:
                if jj in self.boundaTruss:
                    node = jj
                    loaded = io.loadmat('WallData.mat')
                    data = loaded['WallData']
                    diam = (float(data[0,0]['Reinf'][0,0]['d_be_e'][0,self.authorIndex]))
                    area = (((diam)/2)**2*m.pi*int(self.balayout[0]))
                    steel_mat = self.beavertTag
            else:
                continue
            for ii in range(self.totLayers-1):
                self.f.write('element truss '+ str(element) + ' ' + str(node)+ ' ' + str(node+self.nodes_along_width) + ' '+ str(area) + ' '+ str(steel_mat) + '\n')
                node +=self.nodes_along_width
                element +=1
            self.f.write('\n')
            
            self.trussele = element-1
        

        self.f.write('\n')
        print(self.barType)   
    def section6(self): #Section 6: Constraints
        
        self.f.write('''


#####################
puts "Define Constraints"
#####################


''')

        for ii in range(1,self.nodes_along_width+1):
            self.f.write('fix ' + str(ii) + ' 1 1 1 1 1 1\n')
            
            
    def section7(self): #Section 7: Recorders
        
        self.f.write('''


#####################
puts "Define Recorders"
#####################


''')
        self.lastRow = []
        firstRow = []
        for ii in range(1,self.nodes_along_width+1):
            n = str(ii)
            firstRow.append(n)

        for ii in range((self.totLayers)*self.nodes_along_width-self.nodes_along_width+1,(self.totLayers)*self.nodes_along_width+1):
            self.lastRow.append(ii)

        for ii in range(1,self.nodes_along_width+1):
            if ii == 1:
                eleb1 = ii
            elif ii == len(self.blayout):
                elew1 = ii
            elif ii == len(self.blayout) + len(self.wlayout):
                elew2 = ii
            elif ii == self.nodes_along_width:
                eleb2 = ii

        last = []
        for ii in self.lastRow:
            last.append(str(ii))


        self.f.write('recorder Node -file baseReactxcyc.txt  -node {}  -dof 1 reaction\n'.format(' '.join(firstRow)) )                           
        self.f.write('recorder Node -file topDispxcyc.txt    -node {}  -dof 1 disp \n'.format(' '.join(last)))              

        self.f.write('recorder Element -xml "FULLWALL_elementsmat1fib3sig.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 1    fiber 3    stresses\n')
        self.f.write('recorder Element -xml "FULLWALL_elementsmat2fib3sig.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 2    fiber 3    stresses\n')
        self.f.write('recorder Element -xml "FULLWALL_elementsmat3fib3sig.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 3    fiber 3    stresses\n')
        self.f.write('recorder Element -xml "FULLWALL_elementsmat4fib3sig.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 4    fiber 3    stresses\n')
        self.f.write('recorder Element -xml "FULLWALL_elementsmat1fib3eps.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 1    fiber 3    strains\n')
        self.f.write('recorder Element -xml "FULLWALL_elementsmat2fib3eps.xml"  -eleRange 1 ' + str(self.maxEle) + '  material 2   fiber 3    strains\n')
        self.f.write('recorder Element -xml "FULLWALL_elementsmat3fib3eps.xml"  -eleRange 1 ' + str(self.maxEle) + '  material 3   fiber 3    strains\n')
        self.f.write('recorder Element -xml "FULLWALL_elementsmat4fib3eps.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 4    fiber 3    strains\n')
        
        self.f.write('#recorder Element -xml "FIRST3LEVELS_elementsmat1fib3sig.xml"  -eleRange 1 ' + str((self.nodes_along_width-1)*3) + ' material 1    fiber 3    stresses\n')
        self.f.write('#recorder Element -xml "FIRST3LEVELS_elementsmat2fib3sig.xml"  -eleRange 1 ' + str((self.nodes_along_width-1)*3) + ' material 2    fiber 3    stresses\n')
        self.f.write('#recorder Element -xml "FIRST3LEVELS_elementsmat3fib3sig.xml"  -eleRange 1 ' + str((self.nodes_along_width-1)*3) + ' material 3    fiber 3    stresses\n')
        self.f.write('#recorder Element -xml "FIRST3LEVELS_elementsmat4fib3sig.xml"  -eleRange 1 ' + str((self.nodes_along_width-1)*3) + ' material 4    fiber 3    stresses\n')
        self.f.write('#recorder Element -xml "FIRST3LEVELS_elementsmat1fib3eps.xml"  -eleRange 1 ' + str((self.nodes_along_width-1)*3) + ' material 1    fiber 3    strains\n')
        self.f.write('#recorder Element -xml "FIRST3LEVELS_elementsmat2fib3eps.xml"  -eleRange 1 ' + str((self.nodes_along_width-1)*3) + '  material 2   fiber 3    strains\n')
        self.f.write('#recorder Element -xml "FIRST3LEVELS_elementsmat3fib3eps.xml"  -eleRange 1 ' + str((self.nodes_along_width-1)*3) + '  material 3   fiber 3    strains\n')
        self.f.write('#recorder Element -xml "FIRST3LEVELS_elementsmat4fib3eps.xml"  -eleRange 1 ' + str((self.nodes_along_width-1)*3) + ' material 4    fiber 3    strains\n')
        
        self.f.write('recorder Element -xml "trusssig.xml"  -eleRange ' + str(self.maxEle+1) + ' ' + str(self.trussele)+ ' material stress\n')
        self.f.write('recorder Element -xml "trussseps.xml" -eleRange ' + str(self.maxEle+1) + ' ' + str(self.trussele)+ ' material strain\n')
        
        self.f.write('recorder Element -xml "Crack_elementsmat1fib3crack.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 1    fiber 3    cracks\n')
        self.f.write('recorder Element -xml "Crack_elementsmat2fib3crack.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 2    fiber 3    cracks\n')
        self.f.write('recorder Element -xml "Crack_elementsmat3fib3crack.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 3    fiber 3    cracks\n')
        self.f.write('recorder Element -xml "Crack_elementsmat4fib3crack.xml"  -eleRange 1 ' + str(self.maxEle) + ' material 4    fiber 3    cracks\n')
        

    def section8(self): #Section 8: Gravity Load

        self.f.write('''


#####################
puts "Define Gravity"
#####################


''')

        axial_load = -1*self.Walldata[45]/self.nodes_along_width

        lastnode = (self.totLayers)*self.nodes_along_width-self.nodes_along_width+1

        self.f.write('pattern Plain 1 Linear {\n')
        for n in range(self.nodes_along_width):
            self.f.write('load ' +  str(lastnode) + ' 0 ' + str(axial_load) + ' 0 0 0 0\n')
            lastnode +=1

        self.f.write('}')

        self.f.write('''


constraints Plain
numberer RCM
system BandGeneral
test NormDispIncr 1.0e-6 200 ;
algorithm BFGS -count 100
integrator LoadControl 0.1;
analysis Static
analyze 10;

puts "gravity analysis ok..."
loadConst -time 0.0; 


''')
        
    def getPeaks(self):
        
        name = str(self.Walldata[1])
        loaded = io.loadmat('WallData.mat')
        data = loaded['WallData']
        locP = data[0,0]["Loading"][0,0]["CyclicHistory"][0,0][name]
        locF = data[0,0]["ExperimentalData"][0,0][name][0,0]['force']
    

        exForce = []

        for ii in range(len(locF)):
            exForce.append(locF[ii][0])
            
        maxStrength = max(exForce)
        
        peaks = []
        for ii in range(len(locP)):
            peaks.append(locP[ii][0])
            peaks.append(locP[ii][1])   
            
        peaks.append(-1*peaks[-1]*2)
        
        height = float(data[0,0]["Geometry"][0,0]['h_measured'][self.authorIndex,0])
        area = float(data[0,0]["Geometry"][0,0]['Area'][self.authorIndex,0])
        fc = float(data[0,0]["Material"][0,0]['fc'][self.authorIndex,0])
       # with open('onsetValues.txt') as h:
       #     data = [line.split() for line in h]
       #    for ii in range(len(data)):
       #         if data[ii][0] == name:
       #             onset = float(data[ii][1])
                    break
        
       # onset = onset/100*height
       # with open(self.filepath + 'onset.txt', 'w') as h:
       #     h.write(str(onset))
            

        with open(self.filepath + 'maxS.txt', 'w') as h:
            h.write(str(maxStrength))
       
        return peaks
    
    def section9(self): #Section 9: Pushover/cyclic
        
        peakdisp = self.getPeaks()
        for ii in range(len(peakdisp)):
            peakdisp[ii] = str(round(peakdisp[ii],2))
            
            

        self.f.write('''


#####################
puts "Define Analysis"
#####################


''')
        if (self.Walldata[6]-self.Walldata[5]) < 2 :
            moment = 0
            extraForce = 0
            self.mom = 0
        else:
            moment = (self.Walldata[6]-self.Walldata[5])*self.nodes_along_width
            self.mom = moment


        self.f.write('pattern Plain 2 Linear {\n')
        if (self.nodes_along_width %2) == 0 and abs(moment) > 1:
            middle = self.nodeArray[int(self.nodes_along_width/2)-1]
            tot = 0
            for ii in range(int(self.nodes_along_width)):
                tot += abs(middle - self.nodeArray[ii])
            extraForce = moment/tot


            for ii in range(-int(self.nodes_along_width),-int(self.nodes_along_width/2)):
                self.f.write('load ' +  str(self.lastRow[ii]) + ' 1 '+ str(extraForce) + ' 0 0 0 0 ' + '\n')
            for ii in range(-int(self.nodes_along_width/2),0):
                self.f.write('load ' +  str(self.lastRow[ii]) + ' 1 '+ str(-extraForce) + ' 0 0 0 0 ' + '\n')                
            
        elif (self.nodes_along_width %2) != 0 and abs(moment) > 1:
            middle = self.nodeArray[int(self.nodes_along_width/2)]
            tot = 0
            for ii in range(int(self.nodes_along_width/2)-1):
                tot += abs(middle - self.nodeArray[ii])
                
            for ii in range(int(self.nodes_along_width/2),self.nodes_along_width):
                tot += abs(middle - self.nodeArray[ii])
            extraForce = moment/tot
                
            for ii in range(-int(self.nodes_along_width),-int(self.nodes_along_width/2)-1):
                self.f.write('load ' +  str(self.lastRow[ii]) + ' 1 '+ str(extraForce) + ' 0 0 0 0 ' + '\n')
                locInd = ii
                
            self.f.write('load ' +  str(self.lastRow[locInd]+1) + ' 1 0 0 0 0 0 ' + '\n')

            
            for ii in range(-int(self.nodes_along_width/2),0):
                self.f.write('load ' +  str(self.lastRow[ii]) + ' 1 '+ str(-extraForce) + ' 0 0 0 0 ' + '\n')      
            
        elif abs(moment) < 1:
            extraForce = 0
            for ii in range(-int(self.nodes_along_width),0):
                self.f.write('load ' +  str(self.lastRow[ii]) + ' 1 '+ str(extraForce) + ' 0 0 0 0 ' + '\n')                 
            
        
        loadnode = (self.totLayers)*len(self.xNodes)-int(len(self.xNodes)/2)

        self.f.write('}')
        
        if self.cyc == True:
            peaks = peakdisp
            
            lastpeak = str(round(float(peaks[-1])*0.99,2))
            self.f.write('\nset DT [list {}]\n'.format(' '.join(peaks)))
            self.dispIndex = []
            for ii in range(len(peaks)):
                if float(peaks[ii])>0:
                    self.dispIndex.append(peaks[ii])
        else:
            peaks = peakdisp[-1]
            self.f.write('\nset DT [list {}]\n'.format(peaks))
            self.dispIndex = []
            if float(peaks)>0:
                self.dispIndex.append(peaks)
        
        self.f.write('set nodeTag ' + str(loadnode) + '\n')
        self.f.write('''
set dofTag 1
set Dtot  0


proc CyclicDisplace { Ddelta Dnum Node dof tol iter last } {

  constraints Transformation
  numberer RCM
  system UmfPack
  test NormDispIncr $tol $iter 2
  analysis Static
  if {$Ddelta> 0} { set Dincr [expr 0.01]}
  if {$Ddelta< 0} { set Dincr [expr -0.01]}
  for {set ii 1} {$ii <=$Dnum} {incr ii} {
    set u [expr $Ddelta*$ii]
    puts "$ii Cyclic of Displacement, Next Cycle..."
    integrator DisplacementControl $Node $dof $Dincr
    Analysis_Proc [expr int($u/$Dincr)] $last
    
    puts "$ii Cyclic of Displacement, Back to Zero..."
    set Dzero [expr $Dincr*-1]
    integrator DisplacementControl $Node $dof $Dzero
    Analysis_Proc [expr int($u/$Dincr)] $last
  }
}

proc Analysis_Proc { Num last} {
  if {$last < 0} {set loc [expr -1*$last]}
    if {$last > 0} {set loc [expr $last]}
  for {set step 1} {$step <=$Num} {incr step} {
    if {($step > [expr $loc*100])} {
    exit}
    puts "No. $step of Cyclic. Anaylsis KrylovNewton.."
    algorithm KrylovNewton -maxDim 10
    set ok [analyze 1]

    if {$ok != 0} {
    puts "No. $step of Cyclic.Anaylsis Trying Newton .."
    algorithm Newton
    set ok [analyze 1]
    }

    if {$ok != 0} {
        puts "No. $step of Cyclic. Analysis Convergence Failure!"
        exit
    }
  }
}


foreach dt $DT {
    CyclicDisplace $dt 1 ''' + str(loadnode) + ''' 1 1E-5 1000 ''' + lastpeak+ '''
}

puts "All of End"
wipe
exit


''')
        self.f.close()
        
    def createScript(self):
        self.section1()
        self.section2()
        self.section3()
        self.section4()
        self.section5()
        self.section6()
        self.section7()
        self.section8()
        self.section9()
        self.r.write(str(self.nodesAlongWidth) + '\n' + str((len(self.boundTruss)+len(self.webTruss)))+ '\n' +
                     str(self.totNodes)+'\n' + str(self.totEle)+'\n')
        res = []
        for i in self.dispIndex:
            if i not in res:
                res.append(i)
        for i in res: self.r.write(str(i) + ' ')
        self.r.write('\n' + str(self.Walldata[42])+'\n' + str(self.totLayers-1)+'\n' + str(self.Walldata[1])+'\n' + str(self.filepath)+'\n' + str(self.filepathtcl))
        self.r.close()
    

In [ ]:
#walls = [108,109,115,116,117]
walls = [0,2,18,19,33,34,63,80,84,92,94,115,116,117,136,137,1,4,5,64,81,82,83,85,93,98,99,100,101,102,118,119,120,133,135]
#walls = [23]
for ii in walls:
    wall = createTclFile(ii,True)
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(5, 10),linewidth=1) #edgecolor="#04253a")
    plt.scatter(wall.xN,wall.yN,s=6)
    ax.set_title(wall.wallspecimen, fontsize=15)

In [ ]:
wall = createTclFile(18,True)